In [1]:
import requests
import time
from threading import Thread, Lock

NUM_MINERS = 10
BASE_PORT = 3000
TARGET_BLOCKS = 1500  # Total blocks to mine across all miners

# Shared counter and lock
total_blocks_mined = 0
lock = Lock()

# Define mining power per miner (must sum to <= 1.0)
# Format: miner_id (1-based index) : mining power (float)
MINING_POWERS = {
    1: 0.3,
    2: 0.2,
    3: 0.1,
    4: 0.1,
    5: 0.1,
    6: 0.05,
    7: 0.05,
    8: 0.05,
    9: 0.05,
}  # Total = 1.0

BASE_INTERVAL = 1  # baseline seconds between mining attempts at full power (1.0)

def mine_blocks(miner_id, port, mining_power):
    global total_blocks_mined
    blocks_mined = 0
    url = f"http://localhost:{port}/api/mine-transactions"
    
    # Adjust mining interval based on power (higher power = faster mining attempts)
    interval = BASE_INTERVAL / mining_power

    while True:
        with lock:
            if total_blocks_mined >= TARGET_BLOCKS:
                break

        try:
            response = requests.get(url)
            if response.status_code == 200:
                try:
                    data = response.json()
                    if data.get("success"):
                        with lock:
                            if total_blocks_mined < TARGET_BLOCKS:
                                total_blocks_mined += 1
                                blocks_mined += 1
                                print(f"[Miner {miner_id} | Port {port}] ✅ Mined block #{blocks_mined} | Total: {total_blocks_mined}")
                    else:
                        reason = data.get("reason", "Unknown")
                        print(f"[Miner {miner_id} | Port {port}] ❌ Block rejected: {reason}")
                except Exception:
                    print(f"[Miner {miner_id} | Port {port}] ⚠️ No valid JSON in response")
            else:
                print(f"[Miner {miner_id} | Port {port}] ❌ Failed with status: {response.status_code}")
        except Exception as e:
            print(f"[Miner {miner_id} | Port {port}] ❌ Exception: {e}")

        time.sleep(interval)

    print(f"[Miner {miner_id} | Port {port}] 🧮 Finished. Accepted blocks mined: {blocks_mined}")

# Launch threads for miners with assigned mining power
threads = []
for miner_id, power in MINING_POWERS.items():
    port = BASE_PORT + miner_id
    t = Thread(target=mine_blocks, args=(miner_id, port, power))
    t.start()
    threads.append(t)
    time.sleep(0.5)  # Optional: staggered start

# Wait for all threads to finish
for t in threads:
    t.join()

print(f"🧪 Mining simulation complete. 🎯 Total accepted blocks mined: {total_blocks_mined}")


[Miner 3 | Port 3003] ✅ Mined block #1 | Total: 1
[Miner 4 | Port 3004] ✅ Mined block #1 | Total: 2
[Miner 1 | Port 3001] ✅ Mined block #1 | Total: 3
[Miner 2 | Port 3002] ✅ Mined block #1 | Total: 4
[Miner 7 | Port 3007] ✅ Mined block #1 | Total: 5
[Miner 5 | Port 3005] ✅ Mined block #1 | Total: 6
[Miner 6 | Port 3006] ✅ Mined block #1 | Total: 7
[Miner 9 | Port 3009] ✅ Mined block #1 | Total: 8
[Miner 8 | Port 3008] ✅ Mined block #1 | Total: 9
[Miner 1 | Port 3001] ✅ Mined block #2 | Total: 10
[Miner 2 | Port 3002] ✅ Mined block #2 | Total: 11
[Miner 1 | Port 3001] ✅ Mined block #3 | Total: 12
[Miner 4 | Port 3004] ✅ Mined block #2 | Total: 13
[Miner 5 | Port 3005] ✅ Mined block #2 | Total: 14
[Miner 3 | Port 3003] ✅ Mined block #2 | Total: 15
[Miner 1 | Port 3001] ✅ Mined block #4 | Total: 16
[Miner 2 | Port 3002] ✅ Mined block #3 | Total: 17
[Miner 1 | Port 3001] ✅ Mined block #5 | Total: 18
[Miner 7 | Port 3007] ✅ Mined block #2 | Total: 19
[Miner 6 | Port 3006] ✅ Mined block #2 |

[Miner 4 | Port 3004] ✅ Mined block #17 | Total: 160
[Miner 9 | Port 3009] ✅ Mined block #10 | Total: 161
[Miner 6 | Port 3006] ✅ Mined block #10 | Total: 162
[Miner 3 | Port 3003] ✅ Mined block #18 | Total: 163
[Miner 7 | Port 3007] ✅ Mined block #10 | Total: 164
[Miner 2 | Port 3002] ✅ Mined block #33 | Total: 165
[Miner 1 | Port 3001] ✅ Mined block #43 | Total: 166
[Miner 5 | Port 3005] ✅ Mined block #17 | Total: 167
[Miner 1 | Port 3001] ✅ Mined block #44 | Total: 168
[Miner 2 | Port 3002] ✅ Mined block #34 | Total: 169
[Miner 1 | Port 3001] ✅ Mined block #45 | Total: 170
[Miner 4 | Port 3004] ✅ Mined block #18 | Total: 171
[Miner 3 | Port 3003] ✅ Mined block #19 | Total: 172
[Miner 2 | Port 3002] ✅ Mined block #35 | Total: 173
[Miner 1 | Port 3001] ✅ Mined block #46 | Total: 174
[Miner 8 | Port 3008] ✅ Mined block #10 | Total: 175
[Miner 5 | Port 3005] ✅ Mined block #18 | Total: 176
[Miner 1 | Port 3001] ✅ Mined block #47 | Total: 177
[Miner 2 | Port 3002] ✅ Mined block #36 | Tota

[Miner 2 | Port 3002] ✅ Mined block #64 | Total: 315
[Miner 1 | Port 3001] ✅ Mined block #88 | Total: 316
[Miner 3 | Port 3003] ✅ Mined block #32 | Total: 317
[Miner 8 | Port 3008] ✅ Mined block #17 | Total: 318
[Miner 2 | Port 3002] ✅ Mined block #65 | Total: 319
[Miner 4 | Port 3004] ✅ Mined block #32 | Total: 320
[Miner 1 | Port 3001] ✅ Mined block #89 | Total: 321
[Miner 2 | Port 3002] ✅ Mined block #66 | Total: 322
[Miner 5 | Port 3005] ✅ Mined block #33 | Total: 323
[Miner 1 | Port 3001] ✅ Mined block #90 | Total: 324
[Miner 2 | Port 3002] ✅ Mined block #67 | Total: 325
[Miner 1 | Port 3001] ✅ Mined block #91 | Total: 326
[Miner 4 | Port 3004] ✅ Mined block #33 | Total: 327
[Miner 7 | Port 3007] ✅ Mined block #19 | Total: 328
[Miner 1 | Port 3001] ✅ Mined block #92 | Total: 329
[Miner 6 | Port 3006] ✅ Mined block #19 | Total: 330
[Miner 3 | Port 3003] ✅ Mined block #33 | Total: 331
[Miner 2 | Port 3002] ✅ Mined block #68 | Total: 332
[Miner 1 | Port 3001] ✅ Mined block #93 | Tota

[Miner 2 | Port 3002] ✅ Mined block #101 | Total: 469
[Miner 1 | Port 3001] ✅ Mined block #135 | Total: 470
[Miner 5 | Port 3005] ✅ Mined block #47 | Total: 471
[Miner 2 | Port 3002] ✅ Mined block #102 | Total: 472
[Miner 1 | Port 3001] ✅ Mined block #136 | Total: 473
[Miner 7 | Port 3007] ✅ Mined block #26 | Total: 474
[Miner 3 | Port 3003] ✅ Mined block #43 | Total: 475
[Miner 4 | Port 3004] ✅ Mined block #47 | Total: 476
[Miner 1 | Port 3001] ✅ Mined block #137 | Total: 477
[Miner 6 | Port 3006] ✅ Mined block #27 | Total: 478
[Miner 5 | Port 3005] ✅ Mined block #48 | Total: 479
[Miner 2 | Port 3002] ✅ Mined block #103 | Total: 480
[Miner 1 | Port 3001] ✅ Mined block #138 | Total: 481
[Miner 3 | Port 3003] ✅ Mined block #44 | Total: 482
[Miner 8 | Port 3008] ✅ Mined block #25 | Total: 483
[Miner 9 | Port 3009] ✅ Mined block #26 | Total: 484
[Miner 2 | Port 3002] ✅ Mined block #104 | Total: 485
[Miner 1 | Port 3001] ✅ Mined block #139 | Total: 486
[Miner 4 | Port 3004] ✅ Mined block #

[Miner 5 | Port 3005] ✅ Mined block #63 | Total: 623
[Miner 2 | Port 3002] ✅ Mined block #132 | Total: 624
[Miner 1 | Port 3001] ✅ Mined block #184 | Total: 625
[Miner 7 | Port 3007] ✅ Mined block #34 | Total: 626
[Miner 9 | Port 3009] ✅ Mined block #33 | Total: 627
[Miner 4 | Port 3004] ✅ Mined block #61 | Total: 628
[Miner 1 | Port 3001] ✅ Mined block #185 | Total: 629
[Miner 2 | Port 3002] ✅ Mined block #133 | Total: 630
[Miner 3 | Port 3003] ✅ Mined block #56 | Total: 631
[Miner 1 | Port 3001] ✅ Mined block #186 | Total: 632
[Miner 6 | Port 3006] ✅ Mined block #35 | Total: 633
[Miner 4 | Port 3004] ✅ Mined block #62 | Total: 634
[Miner 5 | Port 3005] ✅ Mined block #64 | Total: 635
[Miner 1 | Port 3001] ✅ Mined block #187 | Total: 636
[Miner 2 | Port 3002] ✅ Mined block #134 | Total: 637
[Miner 1 | Port 3001] ✅ Mined block #188 | Total: 638
[Miner 2 | Port 3002] ✅ Mined block #135 | Total: 639
[Miner 1 | Port 3001] ✅ Mined block #189 | Total: 640
[Miner 8 | Port 3008] ✅ Mined block 

[Miner 7 | Port 3007] ✅ Mined block #40 | Total: 776
[Miner 4 | Port 3004] ✅ Mined block #73 | Total: 777
[Miner 2 | Port 3002] ✅ Mined block #174 | Total: 778
[Miner 1 | Port 3001] ✅ Mined block #232 | Total: 779
[Miner 2 | Port 3002] ✅ Mined block #175 | Total: 780
[Miner 9 | Port 3009] ✅ Mined block #39 | Total: 781
[Miner 1 | Port 3001] ✅ Mined block #233 | Total: 782
[Miner 2 | Port 3002] ✅ Mined block #176 | Total: 783
[Miner 5 | Port 3005] ✅ Mined block #77 | Total: 784
[Miner 1 | Port 3001] ✅ Mined block #234 | Total: 785
[Miner 2 | Port 3002] ✅ Mined block #177 | Total: 786
[Miner 1 | Port 3001] ✅ Mined block #235 | Total: 787
[Miner 3 | Port 3003] ✅ Mined block #68 | Total: 788
[Miner 2 | Port 3002] ✅ Mined block #178 | Total: 789
[Miner 1 | Port 3001] ✅ Mined block #236 | Total: 790
[Miner 4 | Port 3004] ✅ Mined block #74 | Total: 791
[Miner 2 | Port 3002] ✅ Mined block #179 | Total: 792
[Miner 1 | Port 3001] ✅ Mined block #237 | Total: 793
[Miner 2 | Port 3002] ✅ Mined bloc

[Miner 5 | Port 3005] ✅ Mined block #88 | Total: 929
[Miner 1 | Port 3001] ✅ Mined block #281 | Total: 930
[Miner 2 | Port 3002] ✅ Mined block #224 | Total: 931
[Miner 1 | Port 3001] ✅ Mined block #282 | Total: 932
[Miner 4 | Port 3004] ✅ Mined block #84 | Total: 933
[Miner 2 | Port 3002] ✅ Mined block #225 | Total: 934
[Miner 9 | Port 3009] ✅ Mined block #45 | Total: 935
[Miner 1 | Port 3001] ✅ Mined block #283 | Total: 936
[Miner 2 | Port 3002] ✅ Mined block #226 | Total: 937
[Miner 1 | Port 3001] ✅ Mined block #284 | Total: 938
[Miner 2 | Port 3002] ✅ Mined block #227 | Total: 939
[Miner 5 | Port 3005] ✅ Mined block #89 | Total: 940
[Miner 1 | Port 3001] ✅ Mined block #285 | Total: 941
[Miner 3 | Port 3003] ✅ Mined block #76 | Total: 942
[Miner 7 | Port 3007] ✅ Mined block #46 | Total: 943
[Miner 2 | Port 3002] ✅ Mined block #228 | Total: 944
[Miner 4 | Port 3004] ✅ Mined block #85 | Total: 945
[Miner 1 | Port 3001] ✅ Mined block #286 | Total: 946
[Miner 2 | Port 3002] ✅ Mined block

[Miner 5 | Port 3005] ✅ Mined block #101 | Total: 1081
[Miner 2 | Port 3002] ✅ Mined block #271 | Total: 1082
[Miner 1 | Port 3001] ✅ Mined block #329 | Total: 1083
[Miner 3 | Port 3003] ✅ Mined block #84 | Total: 1084
[Miner 2 | Port 3002] ✅ Mined block #272 | Total: 1085
[Miner 4 | Port 3004] ✅ Mined block #97 | Total: 1086
[Miner 1 | Port 3001] ✅ Mined block #330 | Total: 1087
[Miner 2 | Port 3002] ✅ Mined block #273 | Total: 1088
[Miner 1 | Port 3001] ✅ Mined block #331 | Total: 1089
[Miner 7 | Port 3007] ✅ Mined block #52 | Total: 1090
[Miner 2 | Port 3002] ✅ Mined block #274 | Total: 1091
[Miner 5 | Port 3005] ✅ Mined block #102 | Total: 1092
[Miner 1 | Port 3001] ✅ Mined block #332 | Total: 1093
[Miner 6 | Port 3006] ✅ Mined block #54 | Total: 1094
[Miner 2 | Port 3002] ✅ Mined block #275 | Total: 1095
[Miner 1 | Port 3001] ✅ Mined block #333 | Total: 1096
[Miner 4 | Port 3004] ✅ Mined block #98 | Total: 1097
[Miner 3 | Port 3003] ✅ Mined block #85 | Total: 1098
[Miner 2 | Port 

[Miner 1 | Port 3001] ✅ Mined block #380 | Total: 1231
[Miner 4 | Port 3004] ✅ Mined block #109 | Total: 1232
[Miner 1 | Port 3001] ✅ Mined block #381 | Total: 1233
[Miner 3 | Port 3003] ✅ Mined block #95 | Total: 1234
[Miner 1 | Port 3001] ✅ Mined block #382 | Total: 1235
[Miner 1 | Port 3001] ✅ Mined block #383 | Total: 1236
[Miner 2 | Port 3002] ✅ Mined block #308 | Total: 1237
[Miner 1 | Port 3001] ✅ Mined block #384 | Total: 1238
[Miner 6 | Port 3006] ✅ Mined block #60 | Total: 1239
[Miner 2 | Port 3002] ✅ Mined block #309 | Total: 1240
[Miner 1 | Port 3001] ✅ Mined block #385 | Total: 1241
[Miner 5 | Port 3005] ✅ Mined block #114 | Total: 1242
[Miner 8 | Port 3008] ✅ Mined block #57 | Total: 1243
[Miner 2 | Port 3002] ✅ Mined block #310 | Total: 1244
[Miner 4 | Port 3004] ✅ Mined block #110 | Total: 1245
[Miner 1 | Port 3001] ✅ Mined block #386 | Total: 1246
[Miner 1 | Port 3001] ✅ Mined block #387 | Total: 1247
[Miner 9 | Port 3009] ✅ Mined block #57 | Total: 1248
[Miner 3 | Por

[Miner 1 | Port 3001] ✅ Mined block #454 | Total: 1381
[Miner 8 | Port 3008] ✅ Mined block #63 | Total: 1382
[Miner 1 | Port 3001] ✅ Mined block #455 | Total: 1383
[Miner 2 | Port 3002] ✅ Mined block #327 | Total: 1384
[Miner 4 | Port 3004] ✅ Mined block #120 | Total: 1385
[Miner 1 | Port 3001] ✅ Mined block #456 | Total: 1386
[Miner 1 | Port 3001] ✅ Mined block #457 | Total: 1387
[Miner 9 | Port 3009] ✅ Mined block #63 | Total: 1388
[Miner 1 | Port 3001] ✅ Mined block #458 | Total: 1389
[Miner 5 | Port 3005] ✅ Mined block #125 | Total: 1390
[Miner 1 | Port 3001] ✅ Mined block #459 | Total: 1391
[Miner 3 | Port 3003] ✅ Mined block #105 | Total: 1392
[Miner 7 | Port 3007] ✅ Mined block #65 | Total: 1393
[Miner 1 | Port 3001] ✅ Mined block #460 | Total: 1394
[Miner 2 | Port 3002] ✅ Mined block #328 | Total: 1395
[Miner 1 | Port 3001] ✅ Mined block #461 | Total: 1396
[Miner 1 | Port 3001] ✅ Mined block #462 | Total: 1397
[Miner 1 | Port 3001] ✅ Mined block #463 | Total: 1398
[Miner 1 | Po

In [2]:
import requests
import json

NODE_PORT = 3001  # change to the port of the node you want to fetch from
BLOCKCHAIN_URL = f"http://localhost:{NODE_PORT}/api/blocks"
OUTPUT_FILE = f"blockchain without rules with power.json"

try:
    response = requests.get(BLOCKCHAIN_URL)
    if response.status_code == 200:
        blockchain = response.json()
        with open(OUTPUT_FILE, 'w') as f:
            json.dump(blockchain, f, indent=2)
        print(f"✅ Blockchain saved to {OUTPUT_FILE}")
    else:
        print(f"❌ Failed to fetch blockchain. Status: {response.status_code}")
except Exception as e:
    print(f"❌ Error fetching blockchain: {e}")


✅ Blockchain saved to blockchain without rules with power.json
